In [1]:
# imports
import pandas as pd
from datetime import datetime
import finnhub
import numpy as np
import matplotlib.pyplot as plot
from sklearn.preprocessing import MinMaxScaler

In [3]:
client = finnhub.Client(api_key='bua9lb748v6q418gd0i0')

def get_candlestick_data(ticker, timeframe, start, end):
    data = client.stock_candles(ticker, timeframe, start, end)
    del data['s']
    df = pd.DataFrame.from_dict(data)
    df['t'] = df['t'].apply(lambda x: datetime.fromtimestamp(x))
    df = df.rename(columns={'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Date', 'v': 'Volume'})
    df.set_index('Date', inplace=True)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    return df

In [6]:
amazon_data = get_candlestick_data('AMZN', 'D', 1601180831, 1603859231)
print(amazon_data)

                            Open         High          Low        Close  \
Date                                                                      
2020-09-27 17:00:00  3148.850098  3175.040039  3117.169922  3174.050049   
2020-09-28 17:00:00  3175.389893  3188.260010  3132.540039  3144.879883   
2020-09-29 17:00:00  3141.139893  3212.879883  3133.989990  3148.729980   
2020-09-30 17:00:00  3208.000000  3224.000000  3172.000000  3221.260010   
2020-10-01 17:00:00  3153.629883  3195.800049  3123.000000  3125.000000   
2020-10-04 17:00:00  3145.840088  3202.530029  3140.850098  3199.199951   
2020-10-05 17:00:00  3165.000000  3182.000000  3090.000000  3099.959961   
2020-10-06 17:00:00  3135.000000  3200.000000  3132.389893  3195.689941   
2020-10-07 17:00:00  3224.989990  3233.290039  3174.989990  3190.550049   
2020-10-08 17:00:00  3210.000000  3288.989990  3197.830078  3286.649902   
2020-10-11 17:00:00  3349.939941  3496.239990  3339.550049  3442.929932   
2020-10-12 17:00:00  3467

In [14]:
open_data = amazon_data.iloc[:, 0:5].values
open_data = open_data.reshape(-1,5)

scaler = MinMaxScaler(feature_range = (0, 1))

open_data = scaler.fit_transform(open_data)
open_data

array([[0.04159314, 0.        , 0.08129353, 0.21558502, 0.20231869],
       [0.12129461, 0.04115807, 0.12728156, 0.13070659, 0.06344483],
       [0.01843867, 0.11780775, 0.13161987, 0.14190948, 0.33178403],
       [0.21922581, 0.1524283 , 0.2453474 , 0.35295509, 0.34639285],
       [0.05594728, 0.06463267, 0.09873737, 0.07286072, 0.46993151],
       [0.0325538 , 0.08558529, 0.1521456 , 0.28876542, 0.11583974],
       [0.0900928 , 0.02166862, 0.        , 0.        , 0.36853797],
       [0.        , 0.07770848, 0.12683231, 0.27855211, 0.21874887],
       [0.27024833, 0.18135121, 0.25429357, 0.26359621, 0.        ],
       [0.225232  , 0.35476329, 0.32263206, 0.54322456, 0.33405182],
       [0.64548469, 1.        , 0.74666409, 0.9979633 , 1.        ],
       [1.        , 0.98798223, 1.        , 1.        , 0.49528736],
       [0.9369651 , 0.90236578, 0.74801036, 0.76745151, 0.51151426],
       [0.4715157 , 0.5630133 , 0.56848787, 0.69453253, 0.39485565],
       [0.68539592, 0.69931478, 0.